In [1]:
import numpy as np
import pandas as pd
import math
import heapq
import time
from datetime import datetime, timedelta, date
import holidays
from calendar import isleap

#### Setting start date

In [2]:
# get days in each month
def get_day_dict(year):
    # Base dictionary for days in each month
    day_dict = {1: 31, 2: 28, 3: 31, 4: 30, 5: 31, 6: 30,
                7: 31, 8: 31, 9: 30, 10: 31, 11: 30, 12: 31}
    # Adjust February for leap years
    if isleap(year):
        day_dict[2] = 29
    return day_dict

In [3]:
year = 2024
month = 8
day = 1
start_date = datetime(year, month, day) 

In [4]:
# holidays in Taiwan
tw_holidays = holidays.TW(years=year)
print(tw_holidays)

{datetime.date(2024, 1, 1): '中華民國開國紀念日', datetime.date(2024, 2, 9): '農曆除夕', datetime.date(2024, 2, 10): '春節', datetime.date(2024, 2, 11): '春節', datetime.date(2024, 2, 12): '春節', datetime.date(2024, 2, 28): '和平紀念日', datetime.date(2024, 4, 4): '兒童節; 清明節', datetime.date(2024, 6, 10): '端午節', datetime.date(2024, 9, 17): '中秋節', datetime.date(2024, 10, 10): '中華民國國慶日', datetime.date(2024, 4, 5): '兒童節（慶祝）', datetime.date(2024, 2, 13): '春節（慶祝）', datetime.date(2024, 2, 14): '春節（慶祝）', datetime.date(2024, 2, 8): '休息日（2024-02-17日起取代）'}


#### Setting Order Number

In [5]:
# 8, 10, 12, 14, 16
order_num = 16

#### Creating Set

In [6]:
day_dict = get_day_dict(year)
Num_L = day_dict[month]

# set of holidays and weekends
HL = set()
for day in range(1, day_dict[month] + 1):
    current_date = date(year, month, day)
    if current_date.weekday() >= 5 or current_date in tw_holidays:  # Check for weekends or holidays
        HL.add(day - 1) # day index is start from zero
print("holiday:", HL)

# dict of day no. and date
L_date = dict()
no = 0 
for l in range(Num_L):
    if l not in HL:
        L_date[no] = l
        no += 1

print(L_date)
L = set(L_date.keys())
print(L)

Num_K = order_num
K = set(range(Num_K))
Num_D = 2
D = set(range(Num_D))

# the number of equipments 
Ed = [10, 10]
# the number of all equipments
Num_E = sum(Ed)
# equipment set
E = set(range(Num_E))
print("E: ", E)

# the number of bearings
Pd = [2, 3]
# the number of all bearings
Num_P = int(sum(np.array(Ed) * np.array(Pd)))
P = set(range(Num_P))
print("P: ", P)

Num_I = 2
I = set(range(Num_I))
print("I: ", I)

# LP-660: 4
# LP-770: 5

holiday: {2, 3, 9, 10, 16, 17, 23, 24, 30}
{0: 0, 1: 1, 2: 4, 3: 5, 4: 6, 5: 7, 6: 8, 7: 11, 8: 12, 9: 13, 10: 14, 11: 15, 12: 18, 13: 19, 14: 20, 15: 21, 16: 22, 17: 25, 18: 26, 19: 27, 20: 28, 21: 29}
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21}
E:  {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}
P:  {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49}
I:  {0, 1}


#### Creating Parameter

In [7]:
# Reading Order Data
# order quantity
with open(f"./data/order/{order_num}/quantity.txt", 'r') as file:
    # Read the content of the file and split it 
    content = file.read()
    q = list(map(int, content.split()))

# order value
with open(f"./data/order/{order_num}/value.txt", 'r') as file:
    # Read the content of the file and split it 
    content = file.read()
    v = list(map(int, content.split()))

# order due date
with open(f"./data/order/{order_num}/due_date.txt", 'r') as file:
    # Read the content of the file and split it 
    content = file.read()
    u = list(map(int, content.split()))
    

In [8]:
# q = [800, 800, 800, 800, 800, 800, 800, 800]
# v = [111942.13, 140985.87, 138375.47, 149479.2, 145684, 151972, 109269.33, 106880.8]
# u = [22, 27, 17, 15, 30, 36, 17, 20] 
r = 0.01
a = 8
b = 100 
M = 9999

#### Adjust due date (to align with zero-started day index)

In [9]:
for i, dd in enumerate(u):
    new_dd = dd - 1 # day index starts from zero 
    u[i] = new_dd

In [10]:
print (u)

[19, 14, 15, 18, 24, 20, 16, 21, 27, 17, 15, 26, 18, 19, 27, 23]


#### Creating Equipment Schedule, Order Schedule and Equipment Usage 

In [11]:
# equipment schedule
eq_schedule = dict()    
for p in P:
    eq_schedule[p] = []

# equipment usage tracker
eq_usage = dict()
for l in L:
    eq_usage[l] = set()

# equipment availablility tracker
eq_avail = dict()
for p in P:
    eq_avail[p] = set()



In [12]:
print(eq_schedule)
print("------")
print(eq_usage)
print("------")
print(eq_avail)

{0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: [], 10: [], 11: [], 12: [], 13: [], 14: [], 15: [], 16: [], 17: [], 18: [], 19: [], 20: [], 21: [], 22: [], 23: [], 24: [], 25: [], 26: [], 27: [], 28: [], 29: [], 30: [], 31: [], 32: [], 33: [], 34: [], 35: [], 36: [], 37: [], 38: [], 39: [], 40: [], 41: [], 42: [], 43: [], 44: [], 45: [], 46: [], 47: [], 48: [], 49: []}
------
{0: set(), 1: set(), 2: set(), 3: set(), 4: set(), 5: set(), 6: set(), 7: set(), 8: set(), 9: set(), 10: set(), 11: set(), 12: set(), 13: set(), 14: set(), 15: set(), 16: set(), 17: set(), 18: set(), 19: set(), 20: set(), 21: set()}
------
{0: set(), 1: set(), 2: set(), 3: set(), 4: set(), 5: set(), 6: set(), 7: set(), 8: set(), 9: set(), 10: set(), 11: set(), 12: set(), 13: set(), 14: set(), 15: set(), 16: set(), 17: set(), 18: set(), 19: set(), 20: set(), 21: set(), 22: set(), 23: set(), 24: set(), 25: set(), 26: set(), 27: set(), 28: set(), 29: set(), 30: set(), 31: set(), 32: set(), 33: set(

#### Creating Orders

In [13]:
orders = {}
for k in K:
    orders[k] = {}
    orders[k]['qty'] = q[k] 
    orders[k]['due'] = u[k]
    orders[k]['value'] = v[k]
    orders[k]['stage'] = []
    for i in I: 
        for j in [0, 1]:
            detail = {}
            detail['id'] = j
            if i == 0:
                detail['side'] = "A"
            if i == 1:
                detail['side'] = "B"
                
            if j == 0:
                detail['process_t'] = a
            else:
                detail['process_t'] = math.ceil(q[k] / b)
    
            orders[k]['stage'].append(detail)

In [14]:
orders

{0: {'qty': 1900,
  'due': 19,
  'value': 12823,
  'stage': [{'id': 0, 'side': 'A', 'process_t': 8},
   {'id': 1, 'side': 'A', 'process_t': 19},
   {'id': 0, 'side': 'B', 'process_t': 8},
   {'id': 1, 'side': 'B', 'process_t': 19}]},
 1: {'qty': 1300,
  'due': 14,
  'value': 33358,
  'stage': [{'id': 0, 'side': 'A', 'process_t': 8},
   {'id': 1, 'side': 'A', 'process_t': 13},
   {'id': 0, 'side': 'B', 'process_t': 8},
   {'id': 1, 'side': 'B', 'process_t': 13}]},
 2: {'qty': 1200,
  'due': 15,
  'value': 27368,
  'stage': [{'id': 0, 'side': 'A', 'process_t': 8},
   {'id': 1, 'side': 'A', 'process_t': 12},
   {'id': 0, 'side': 'B', 'process_t': 8},
   {'id': 1, 'side': 'B', 'process_t': 12}]},
 3: {'qty': 1400,
  'due': 18,
  'value': 93904,
  'stage': [{'id': 0, 'side': 'A', 'process_t': 8},
   {'id': 1, 'side': 'A', 'process_t': 14},
   {'id': 0, 'side': 'B', 'process_t': 8},
   {'id': 1, 'side': 'B', 'process_t': 14}]},
 4: {'qty': 1900,
  'due': 24,
  'value': 44398,
  'stage': [{'i

#### Sorting Orders

In [15]:
order_seq = []
for k in K:
    qty = orders[k]['qty']  # Use -qty as priority, because we want to make larger quantity order has higher priority
    heapq.heappush(order_seq, (-qty, k))

In [16]:
order_seq

[(-2000, 9),
 (-1900, 0),
 (-1900, 13),
 (-1700, 7),
 (-1900, 4),
 (-1300, 11),
 (-1800, 12),
 (-1300, 1),
 (-1200, 8),
 (-1400, 3),
 (-1800, 10),
 (-800, 5),
 (-1200, 2),
 (-1200, 6),
 (-1600, 14),
 (-800, 15)]

#### Scheduling 

In [17]:
# record cpu time
start_time = time.time()

In [18]:
# check each equipment and bearing and find available candidate equipment
def _check_aval(p, start_t, end_t, cand_eq):
    # check bearing limit 
    for t_p in range(start_t, end_t):
        if len(eq_usage[t_p]) >= 50:
            return cand_eq
            
    # check time availability
    for t_p in range(start_t, end_t):
        if t_p in eq_avail[p]:
            return cand_eq

    heapq.heappush(cand_eq, (start_t, p))
    return cand_eq
        

In [19]:
while order_seq:
    _, k = heapq.heappop(order_seq)
    for task in orders[k]['stage']:
        # stage 0, we search the feasible start slot for stage 0 at the same time
        if task['id'] == 0:
            # (start_time, bearing)
            cand_equip = []
            # check equipment and bearing availability
            for p in P:
                for l in range(len(L) - a + 1):
                     cand_equip = _check_aval(p, l, l + a , cand_equip)
    
            # choose the start time slot with min start time
            if len(cand_equip) > 0:
                st, br = heapq.heappop(cand_equip)
                # update equipment schedule (order_id, side, stage, starttime, endtime)
                # stage 0
                eq_schedule[br].append((k, task['side'], task['id'], st, st + a - 1))
                
                # update equipment usage (bearing) 
                # update equipment availability (time)
                for l in range(st, st + a):
                    eq_usage[l].add(br)
                    eq_avail[br].add(l)

                # record stage 1 end_time
                end_time_s1 = st + a - 1
                
            else:
                print("!!![stage 0] cannot find feasible start time slot!!!")

        # stage 2
        if task['id'] == 1:
            # each processing time seemed as a task
            for _ in range(task['process_t']):
 
                # (start_time, equipment, bearing)
                cand_equip = []
                # check equipment and bearing availability
                for p in P:
                    for l in range(len(L)):
                        # stage 2 start time should be later than stage 1
                        if l > end_time_s1: 
                            # print(f"order {k}, side {task['side']}, stage {task['id']}: time {l}")
                            cand_equip = _check_aval(p, l, l + 1, cand_equip)

                # choose the start time slot with min start time
                if len(cand_equip) > 0:
                    st, br = heapq.heappop(cand_equip)
                    # update equipment schedule (order_id, side, stage, starttime, endtime)
                    # stage 1
                    eq_schedule[br].append((k, task['side'], task['id'], st, st))
                    # update equipment usage (equipment, bearing) 
                    # update equipment availability (time)
                    eq_usage[st].add(br)
                    eq_avail[br].add(st)
                                
                else:
                    print("!!![stage 1] cannot find feasible start time slot!!!")
    
                

In [20]:
# End the timer 
cpu_time = time.time() - start_time

In [21]:
print(eq_schedule)
print("------")
print(eq_usage)
print("------")
print(eq_avail)

{0: [(9, 'A', 0, 0, 7), (9, 'A', 1, 8, 8), (0, 'A', 1, 9, 9), (4, 'B', 1, 10, 10), (13, 'B', 1, 11, 11), (12, 'A', 1, 12, 12), (7, 'B', 1, 13, 13), (3, 'A', 1, 14, 14), (11, 'A', 1, 15, 15), (6, 'A', 1, 16, 16), (5, 'A', 1, 17, 17)], 1: [(9, 'A', 1, 8, 8), (9, 'B', 0, 0, 7), (0, 'A', 1, 9, 9), (4, 'B', 1, 10, 10), (13, 'B', 1, 11, 11), (12, 'A', 1, 12, 12), (7, 'B', 1, 13, 13), (3, 'A', 1, 14, 14), (11, 'A', 1, 15, 15), (6, 'A', 1, 16, 16), (5, 'A', 1, 17, 17)], 2: [(9, 'A', 1, 8, 8), (0, 'A', 0, 0, 7), (0, 'A', 1, 9, 9), (4, 'B', 1, 10, 10), (13, 'B', 1, 11, 11), (12, 'A', 1, 12, 12), (7, 'B', 1, 13, 13), (3, 'A', 1, 14, 14), (11, 'A', 1, 15, 15), (6, 'A', 1, 16, 16), (5, 'B', 1, 17, 17)], 3: [(9, 'A', 1, 8, 8), (0, 'A', 1, 9, 9), (0, 'B', 0, 0, 7), (4, 'B', 1, 10, 10), (13, 'B', 1, 11, 11), (12, 'A', 1, 12, 12), (7, 'B', 1, 13, 13), (3, 'A', 1, 14, 14), (11, 'A', 1, 15, 15), (6, 'A', 1, 16, 16), (5, 'B', 1, 17, 17)], 4: [(9, 'A', 1, 8, 8), (0, 'A', 1, 9, 9), (4, 'A', 0, 0, 7), (4, 'B

#### Calculate Objective Function

In [22]:
# create a dictionary to store order end time
order_max_end_time = dict()
for p in P:
    for order_id, side, stage, start_t, end_t in eq_schedule[p]:
        if stage == 1:
            if order_id not in order_max_end_time:
                order_max_end_time[order_id] = 0
            if L_date[end_t] > order_max_end_time[order_id]:
                order_max_end_time[order_id] = L_date[end_t]

print(order_max_end_time)

{9: 12, 0: 13, 4: 14, 13: 15, 12: 18, 7: 19, 3: 20, 11: 21, 6: 22, 5: 25, 10: 15, 14: 19, 15: 25, 1: 20, 8: 22, 2: 21}


In [23]:
u

[19, 14, 15, 18, 24, 20, 16, 21, 27, 17, 15, 26, 18, 19, 27, 23]

In [24]:
# calculate tardiness for each order
order_tard = {}
for order_id, max_end_time in order_max_end_time.items():
    tard = max(max_end_time - u[order_id], 0)
    order_tard[order_id] = tard

print(order_tard)

{9: 0, 0: 0, 4: 0, 13: 0, 12: 0, 7: 0, 3: 2, 11: 0, 6: 6, 5: 5, 10: 0, 14: 0, 15: 2, 1: 6, 8: 0, 2: 6}


In [25]:
# calculate week tardiness and objective
obj = 0 
order_tard_week = {}
for key, value in order_tard.items():
    order_tard_week[key] = math.ceil(value / 7)
    obj += v[key] * order_tard_week[key] * r

In [26]:
order_tard_week

{9: 0,
 0: 0,
 4: 0,
 13: 0,
 12: 0,
 7: 0,
 3: 1,
 11: 0,
 6: 1,
 5: 1,
 10: 0,
 14: 0,
 15: 1,
 1: 1,
 8: 0,
 2: 1}

In [27]:
print(obj)
print(f"cpu time: {cpu_time:.2f} seconds")

3380.89
cpu time: 0.62 seconds


#### Generating Excel

In [28]:
# Create dataframe 
tuple_list = []
# Two types of machines
p = 0
while p < len(P):
    for d in [0, 1]: 
        for e in range(Ed[d]):
            if d > 0:
                e += sum(Ed[:d])
            for _ in range(Pd[d]):
                tuple_list.append((f"Equipment {e}", f"Bearing {p}" ))
                p+=1
                

col_name = pd.MultiIndex.from_tuples(tuple_list, names=['Equipment', 'Bearing'])
# using all date
df = pd.DataFrame(index=[f"Day {day}" for day in range(Num_L)], columns = col_name)

In [29]:
df

Equipment Equipment 0           Equipment 1           Equipment 2            \
Bearing     Bearing 0 Bearing 1   Bearing 2 Bearing 3   Bearing 4 Bearing 5   
Day 0             NaN       NaN         NaN       NaN         NaN       NaN   
Day 1             NaN       NaN         NaN       NaN         NaN       NaN   
Day 2             NaN       NaN         NaN       NaN         NaN       NaN   
Day 3             NaN       NaN         NaN       NaN         NaN       NaN   
Day 4             NaN       NaN         NaN       NaN         NaN       NaN   
Day 5             NaN       NaN         NaN       NaN         NaN       NaN   
Day 6             NaN       NaN         NaN       NaN         NaN       NaN   
Day 7             NaN       NaN         NaN       NaN         NaN       NaN   
Day 8             NaN       NaN         NaN       NaN         NaN       NaN   
Day 9             NaN       NaN         NaN       NaN         NaN       NaN   
Day 10            NaN       NaN         NaN       NaN         NaN       NaN   
Day 11            NaN       NaN         NaN       NaN         NaN       NaN   
Day 12            NaN       NaN         NaN       NaN         NaN       NaN   
Day 13            NaN       NaN         NaN       NaN         NaN       NaN   
Day 14            NaN       NaN         NaN       NaN         NaN       NaN   
Day 15            NaN       NaN         NaN       NaN         NaN       NaN   
Day 16            NaN       NaN         NaN       NaN         NaN       NaN   
Day 17            NaN       NaN         NaN       NaN         NaN       NaN   
Day 18            NaN       NaN         NaN       NaN         NaN       NaN   
Day 19            NaN       NaN         NaN       NaN         NaN       NaN   
Day 20            NaN       NaN         NaN       NaN         NaN       NaN   
Day 21            NaN       NaN         NaN       NaN         NaN       NaN   
Day 22            NaN       NaN         NaN       NaN         NaN       NaN   
Day 23            NaN       NaN         NaN       NaN         NaN       NaN   
Day 24            NaN       NaN         NaN       NaN         NaN       NaN   
Day 25            NaN       NaN         NaN       NaN         NaN       NaN   
Day 26            NaN       NaN         NaN       NaN         NaN       NaN   
Day 27            NaN       NaN         NaN       NaN         NaN       NaN   
Day 28            NaN       NaN         NaN       NaN         NaN       NaN   
Day 29            NaN       NaN         NaN       NaN         NaN       NaN   
Day 30            NaN       NaN         NaN       NaN         NaN       NaN   

Equipment Equipment 3           Equipment 4            ... Equipment 16  \
Bearing     Bearing 6 Bearing 7   Bearing 8 Bearing 9  ...   Bearing 40   
Day 0             NaN       NaN         NaN       NaN  ...          NaN   
Day 1             NaN       NaN         NaN       NaN  ...          NaN   
Day 2             NaN       NaN         NaN       NaN  ...          NaN   
Day 3             NaN       NaN         NaN       NaN  ...          NaN   
Day 4             NaN       NaN         NaN       NaN  ...          NaN   
Day 5             NaN       NaN         NaN       NaN  ...          NaN   
Day 6             NaN       NaN         NaN       NaN  ...          NaN   
Day 7             NaN       NaN         NaN       NaN  ...          NaN   
Day 8             NaN       NaN         NaN       NaN  ...          NaN   
Day 9             NaN       NaN         NaN       NaN  ...          NaN   
Day 10            NaN       NaN         NaN       NaN  ...          NaN   
Day 11            NaN       NaN         NaN       NaN  ...          NaN   
Day 12            NaN       NaN         NaN       NaN  ...          NaN   
Day 13            NaN       NaN         NaN       NaN  ...          NaN   
Day 14            NaN       NaN         NaN       NaN  ...          NaN   
Day 15            NaN       NaN         NaN       NaN  ...          NaN   
Day 16            NaN       NaN         Na

In [30]:
# Fill Scheduling DataFrame
for p in P:
    for k, side, stage, start_t, end_t in eq_schedule[p]:
        for l in range(start_t, end_t + 1):                                                
            col_mask = df.columns.get_level_values(1) == f"Bearing {p}"
            df.loc[f"Day {L_date[l]}", col_mask] = f"Order {k} Side {side} Stage {stage+1}"
            # print(f"Order {k} Side {i} stage {j} is printing on Equipment {e} Bearing {p} at time {l}")

In [31]:
df

Equipment              Equipment 0                           \
Bearing                  Bearing 0                Bearing 1   
Day 0       Order 9 Side A Stage 1   Order 9 Side B Stage 1   
Day 1       Order 9 Side A Stage 1   Order 9 Side B Stage 1   
Day 2                          NaN                      NaN   
Day 3                          NaN                      NaN   
Day 4       Order 9 Side A Stage 1   Order 9 Side B Stage 1   
Day 5       Order 9 Side A Stage 1   Order 9 Side B Stage 1   
Day 6       Order 9 Side A Stage 1   Order 9 Side B Stage 1   
Day 7       Order 9 Side A Stage 1   Order 9 Side B Stage 1   
Day 8       Order 9 Side A Stage 1   Order 9 Side B Stage 1   
Day 9                          NaN                      NaN   
Day 10                         NaN                      NaN   
Day 11      Order 9 Side A Stage 1   Order 9 Side B Stage 1   
Day 12      Order 9 Side A Stage 2   Order 9 Side A Stage 2   
Day 13      Order 0 Side A Stage 2   Order 0 Side A Stage 2   
Day 14      Order 4 Side B Stage 2   Order 4 Side B Stage 2   
Day 15     Order 13 Side B Stage 2  Order 13 Side B Stage 2   
Day 16                         NaN                      NaN   
Day 17                         NaN                      NaN   
Day 18     Order 12 Side A Stage 2  Order 12 Side A Stage 2   
Day 19      Order 7 Side B Stage 2   Order 7 Side B Stage 2   
Day 20      Order 3 Side A Stage 2   Order 3 Side A Stage 2   
Day 21     Order 11 Side A Stage 2  Order 11 Side A Stage 2   
Day 22      Order 6 Side A Stage 2   Order 6 Side A Stage 2   
Day 23                         NaN                      NaN   
Day 24                         NaN                      NaN   
Day 25      Order 5 Side A Stage 2   Order 5 Side A Stage 2   
Day 26                         NaN                      NaN   
Day 27                         NaN                      NaN   
Day 28                         NaN                      NaN   
Day 29                         NaN                      NaN   
Day 30                         NaN                      NaN   

Equipment              Equipment 1                           \
Bearing                  Bearing 2                Bearing 3   
Day 0       Order 0 Side A Stage 1   Order 0 Side B Stage 1   
Day 1       Order 0 Side A Stage 1   Order 0 Side B Stage 1   
Day 2                          NaN                      NaN   
Day 3                          NaN                      NaN   
Day 4       Order 0 Side A Stage 1   Order 0 Side B Stage 1   
Day 5       Order 0 Side A Stage 1   Order 0 Side B Stage 1   
Day 6       Order 0 Side A Stage 1   Order 0 Side B Stage 1   
Day 7       Order 0 Side A Stage 1   Order 0 Side B Stage 1   
Day 8       Order 0 Side A Stage 1   Order 0 Side B Stage 1   
Day 9                          NaN                      NaN   
Day 10                         NaN                      NaN   
Day 11      Order 0 Side A Stage 1   Order 0 Side B Stage 1   
Day 12      Order 9 Side A Stage 2   Order 9 Side A Stage 2   
Day 13      Order 0 Side A Stage 2   Order 0 Side A Stage 2   
Day 14      Order 4 Side B Stage 2   Order 4 Side B Stage 2   
Day 15     Order 13 Side B Stage 2  Order 13 Side B Stage 2   
Day 16                         NaN                      NaN   
Day 17                         NaN                      NaN   
Day 18     Order 12 Side A Stage 2  Order 12 Side A Stage 2   
Day 19      Order 7 Side B Stage 2   Order 7 Side B Stage 2   
Day 20      Order 3 Side A Stage 2   Order 3 Side A Stage 2   
Day 21     Order 11 Side A Stage 2  Order 11 Side A Stage 2   
Day 22      Order 6 Side A Stage 2   Order 6 Side A Stage 2   
Day 23                         NaN                      NaN   
Day 24                         NaN                      NaN   
Day 25      Order 5 Side B Stage 2   Order 5 Side B Stage 2   
Day 26                         NaN                      NaN   
Day 27                         NaN                      NaN   
Day 28                        

In [32]:
# Replace day no. with date 
df.index = df.index.map(lambda x: (start_date + timedelta(days=int(x.split()[1]))).strftime('%m/%d'))

In [33]:
# Use color to represent different orders
color_pairs = {
    'Order 0': ('#1f77b4', '#aec7e8'),
    'Order 1': ('#ff7f0e', '#ffbb78'),
    'Order 2': ('#2ca02c', '#98df8a'),
    'Order 3': ('#d62728', '#ff9896'),
    'Order 4': ('#9467bd', '#c5b0d5'),
    'Order 5': ('#8c564b', '#c49c94'),
    'Order 6': ('#e377c2', '#f7b6d2'),
    'Order 7': ('#7f7f7f', '#c7c7c7'),
    'Order 8': ('#bcbd22', '#dbdb8d'),
    'Order 9': ('#17becf', '#9edae5'),
    'Order 10': ('#393b79', '#5254a3'),
    'Order 11': ('#637939', '#8ca252'),
    'Order 12': ('#8c6d31', '#bd9e39'),
    'Order 13': ('#843c39', '#ad494a'),
    'Order 14': ('#7b4173', '#a55194'),
    'Order 15': ('#17becf', '#98df8a'),
}

In [34]:
# Save the Excel
with pd.ExcelWriter(f"./schedule_result/result/{order_num}/scheduling_algorithm_order_{order_num}_practical.xlsx") as writer:
    df.to_excel(writer, sheet_name="Schedule")
    
    # Access the xlsxwriter workbook and worksheet objects
    workbook  = writer.book
    worksheet = writer.sheets["Schedule"]
    
    # Set the width of each column
    for col_num, col in enumerate(df.columns, start=1):
        worksheet.set_column(col_num, col_num, 25)

    # set color for each order 
    for col_num, col in enumerate(df.columns, start=1):
        for row_num, value in enumerate(df[col], start=3):
            if pd.notna(value):
                # split the value, which is Order k Side i Stage j
                split_str = value.split()
                order_key = split_str[0] + " " + split_str[1]

                if "A" in split_str :
                    color = color_pairs[order_key][0]
                else:
                    color = color_pairs[order_key][1]
                          
                worksheet.write(row_num, col_num, value, workbook.add_format({'bg_color': color}))
